In [1]:
#path = '/Users/amirnazif/Downloads/combined_470.csv'
path = '/Users/simonbian/大三下/CS470/project/combined.csv'
import pandas as pd
df = pd.read_csv(path)
df_small = df.copy()

/var/folders/0p/bfk7s5314wzfkww172k0x4280000gn/T/ipykernel_38163/1587522097.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [2]:
df_small['GPA'] = df_small['GPA'].str.replace("GPA ", "")
df_small[['Status','Date']]=df_small['Status_Date'].str.split(' ', n=1, expand=True)

In [3]:
df_small['Date_Added'] = pd.to_datetime(df_small['Date_Added'])

In [4]:
df_small[['Major','School']]=df_small['Major_School'].str.split(', ', n=1, expand=True)

In [5]:
df_small = df_small.drop(['Date','Major_School','Description','Status_Date','Attribute_1',"Attribute_2","Attribute_3"],axis=1)

In [6]:
import numpy as np
def extract_scores(row):
    if pd.isna(row):
        return np.nan, np.nan, np.nan  # Handle NA entries
    parts = row.split(';')
    gre_q, gre_v, gre_aw = np.nan, np.nan, np.nan
    for part in parts:
        if 'GRE ' in part and 'V' not in part and 'AW' not in part:
            gre_q = part.split()[-1]
        elif 'GRE V' in part:
            gre_v = part.split()[-1]
        elif 'GRE AW' in part:
            gre_aw = part.split()[-1]
    return gre_q, gre_v, gre_aw
df_small['GRE_Q'], df_small['GRE_V'], df_small['GRE_AW'] = zip(*df_small['GRE_Stats'].apply(extract_scores))

# Convert the new columns to numeric, handling errors
df_small['GRE_Q'] = pd.to_numeric(df_small['GRE_Q'], errors='coerce')
df_small['GRE_V'] = pd.to_numeric(df_small['GRE_V'], errors='coerce')
df_small['GRE_AW'] = pd.to_numeric(df_small['GRE_AW'], errors='coerce')

In [7]:
df_small = df_small.drop(["GRE_Stats"],axis=1)

In [8]:
df_small

,Date_Added,GPA,Degree,Citizenship,Status,Major,School,GRE_Q,GRE_V,GRE_AW
0,2024-01-29,NaN,Masters,International,Rejected,Education Policy,University Of Wisconsin-Madison,NaN,NaN,NaN
1,2024-01-29,4.00,PhD,International,Accepted,Computer Science,University Of Illinois,NaN,NaN,NaN
2,2024-01-29,NaN,PhD,American,Rejected,Astronomy,Indiana University Bloomington,NaN,NaN,NaN
3,2024-01-29,3.00,PhD,International,Accepted,Physics,Penn State University,NaN,NaN,NaN
4,2024-01-29,NaN,PhD,American,Accepted,Integrated Program In Biochemistry (IPiB),University Of Wisconsin-Madison,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
847720,2006-02-11,NaN,NaN,NaN,Accepted,Advertising,"University of Texas, Austin (UT Austin)",NaN,NaN,NaN
847721,2006-02-11,NaN,NaN,NaN,Accepted,ECE,"University Of Wisconsin, Madison",NaN,NaN,NaN
847722,2006-02-11,NaN,NaN,NaN,Accepted,English,Cornell University,NaN,NaN,NaN
847723,2006-02-11,NaN,PhD,NaN,Accepted,Phyics,New York University (NYU),NaN,NaN,NaN


In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Example dataframe loading (replace with your actual data loading code)
# df = pd.read_csv('your_data.csv')

# Sample data creation for illustration (replace df with your actual DataFrame)
df = df_small

# Convert 'Status' to binary
#df['Status'] = df['Status'].map({'Accepted': 1, 'Rejected': 0})

# Define preprocessing for numeric columns (impute then scale)
numeric_features = ['GRE_Q',"GRE_V","GRE_AW",'GPA']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))]#,
    #('scaler', StandardScaler())])

# Define preprocessing for categorical columns (impute then one-hot encode)
categorical_features = ['Degree', 'Citizenship']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Create the preprocessing and training pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', DecisionTreeClassifier(max_depth = 6, random_state=0))])

# Split the dataset
df = df.fillna(df.select_dtypes(include='number').mean()).dropna(axis=0, how='any', subset=df.select_dtypes(include='object').columns)

X = df.drop('Status', axis=1)
y = df['Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Train the model
pipeline.fit(X, y)

# Predict and evaluate the model
y_pred = pipeline.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")


SyntaxError: invalid syntax (4115733282.py, line 26)

In [13]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

# Assuming 'pipeline' is your trained Pipeline containing the DecisionTreeClassifier
# Extract the decision tree model from the pipeline
decision_tree_model = pipeline.named_steps['classifier']

# Visualize the decision tree
plt.figure(figsize=(80,20))  # Set the figure size for better readability
plot_tree(decision_tree_model, 
          filled=True, 
          feature_names=pipeline.named_steps['preprocessor'].transformers_[0][2] + 
                        list(pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out()), fontsize = 12)
plt.savefig('decision_tree_visualization.svg', format='svg', bbox_inches='tight')
plt.close()

In [16]:
X

,Date_Added,GPA,Degree,Citizenship,Major,School,GRE_Q,GRE_V,GRE_AW
1,2024-01-29,4.00,PhD,International,Computer Science,University Of Illinois,232.416261,218.151493,4.773105
3,2024-01-29,3.00,PhD,International,Physics,Penn State University,232.416261,218.151493,4.773105
6,2024-01-29,3.92,PhD,International,EECS,MIT,232.416261,218.151493,4.773105
7,2024-01-29,3.90,PhD,International,Statistics,Duke University,170.000000,164.000000,3.500000
12,2024-01-29,3.40,PhD,American,Clinical Psychology,University Of Michigan (Ann Arbor),232.416261,218.151493,4.773105
...,...,...,...,...,...,...,...,...,...
788209,2010-01-12,3.40,PhD,American,Biological Sciences (Neuroscience),Carnegie Mellon,800.000000,690.000000,5.000000
788210,2010-01-12,3.60,Masters,American,Master Of Public Administration (MPA),London School Of Economics (LSE),710.000000,510.000000,4.773105
788211,2010-01-12,3.60,Masters,American,International Public Policy,University College London,710.000000,510.000000,4.773105
788213,2010-01-12,3.86,PhD,American,Neuroscience,"University Of California, San Francisco (UCSF)",800.000000,610.000000,4.000000


In [26]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['GRE_Q', 'GRE_V', 'GRE_AW',
                                                   'GPA']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Degree', 'Citizenship'])])),
                ('classifier',
                 DecisionTreeClassifier(max_depth=6, random_state=0))])

In [30]:
from sklearn.tree import export_text
# Extract the decision tree model from the pipeline
decision_tree_model = pipeline.named_steps['classifier']
# Assuming 'pipeline' is your fitted pipeline and 'numeric_features' is the list of numeric features
scaler = pipeline.named_steps['preprocessor'].transformers_[0][1].named_steps['scaler']


def get_feature_names(column_transformer):
    """Get feature names from a ColumnTransformer."""
    output_features = []

    # Loop through the transformers
    for name, pipe, features in column_transformer.transformers_:
        if name == 'remainder':  # Skip the 'remainder' transformer
            continue
        if hasattr(pipe, 'steps'):  # If it's a pipeline
            inner_transform = pipe.steps[-1][1]
            if hasattr(inner_transform, 'get_feature_names_out'):
                # If the inner transformer has `get_feature_names_out`, use it
                names = inner_transform.get_feature_names_out(features)
            elif hasattr(inner_transform, 'get_feature_names'):
                # If it has `get_feature_names`, use it
                names = inner_transform.get_feature_names(features)
            else:
                # Otherwise, just use the given feature names
                names = features
        else:
            if hasattr(pipe, 'get_feature_names_out'):
                # Directly get feature names from the transformer
                names = pipe.get_feature_names_out(features)
            elif hasattr(pipe, 'get_feature_names'):
                names = pipe.get_feature_names(features)
            else:
                names = features
        # Add the feature names to our output list
        output_features.extend(names)

    return output_features

# After fitting the pipeline, get the feature names
feature_names = get_feature_names(pipeline.named_steps['preprocessor'])

# Use these feature names with `export_text`
tree_rules = export_text(decision_tree_model, feature_names=feature_names)
print(tree_rules)




|--- Degree_PhD <= 0.50
|   |--- Citizenship_American <= 0.50
|   |   |--- GPA <= -0.37
|   |   |   |--- GRE_AW <= -0.04
|   |   |   |   |--- GRE_Q <= -0.45
|   |   |   |   |   |--- GPA <= -1.89
|   |   |   |   |   |   |--- class: Rejected
|   |   |   |   |   |--- GPA >  -1.89
|   |   |   |   |   |   |--- class: Accepted
|   |   |   |   |--- GRE_Q >  -0.45
|   |   |   |   |   |--- GPA <= -1.62
|   |   |   |   |   |   |--- class: Accepted
|   |   |   |   |   |--- GPA >  -1.62
|   |   |   |   |   |   |--- class: Accepted
|   |   |   |--- GRE_AW >  -0.04
|   |   |   |   |--- GPA <= -1.36
|   |   |   |   |   |--- GRE_AW <= 0.00
|   |   |   |   |   |   |--- class: Accepted
|   |   |   |   |   |--- GRE_AW >  0.00
|   |   |   |   |   |   |--- class: Accepted
|   |   |   |   |--- GPA >  -1.36
|   |   |   |   |   |--- GRE_Q <= -0.51
|   |   |   |   |   |   |--- class: Accepted
|   |   |   |   |   |--- GRE_Q >  -0.51
|   |   |   |   |   |   |--- class: Accepted
|   |   |--- GPA >  -0.37
|   |   